# Creating a reference dataframe for Yelp data

Since we did not have available rental units for every single zipcode in NYC, there were some zipcodes which we do not have Yelp data on.

The strategy for these zipcodes was to estimate values for these Yelp metrics:

- Yelp "$" ratings
- Yelp user ratings
- Yelp number of reviews
- Sentiments (based on sentiment analysis of review text)

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('/Users/peidacai/Desktop/rent_yelp_taxi_df.pkl', 'r') as f:
    rent_yelp_taxi = pickle.load(f)

In [3]:
rent_yelp_taxi.head(1)

,Full_Street_address,SF_avail,Price_per_sf_per_year,Prop_desc,Latitude,Longitude,City,Zipcode,Position,block_id,...,Review_min,Sentiment_mean,Sentiment_std,Sentiment_max,Sentiment_min,dropoff_count,dropoff_hr_0600_1200,dropoff_hr_1200_1800,dropoff_hr_1800_2359,dropoff_hr_2359_0600
0,"327 W 57th St, New York, NY 10019",3840,118.75,"(""Amazing Location! Perfect for Restaurant or ...",40.767498,-73.984111,New York,10019,"[-73.9841113, 40.7674983]",15000US360610139007,...,41,0.211583,0.06027,0.317372,0.100961,1681,369.0,549.0,564.0,199.0


In [4]:
# Load in a NYC Zipcode list

nyc_zip = pd.read_csv('../part-02/assets/data/NYC_zipcodes/nyc_zip.txt', header=None)
nyc_zip.head()

,0,1
0,0,10001
1,1,10002
2,2,10003
3,3,10004
4,4,10005


In [5]:
# Pivoting to get only input features for the model (Yelp only)

mean_df = pd.pivot_table(rent_yelp_taxi, 
                         values = ['Cost_mean', 'Cost_std',
                                   'Rate_mean', 'Rate_std',
                                   'Review_mean', 'Review_std',
                                   'Sentiment_mean'
                                  ], 
                         index = ['Zipcode'])

In [6]:
# Create a list of zipcodes which we do not have rental locations for
# Without rental locations, there was no data on Yelp for these zipcodes

nan_zip = []

for i in nyc_zip[1]:
    if i not in mean_df.index:
        nan_zip.append(i)

## Method 1: Create normally distributed random yelp values for missing zipcodes

In [7]:
# Create an empty dictionary to with columns in mean_df as keys
estimation = {}

# Iterating through the mean_df to get mean and stdev of each column
for i in mean_df.columns:
    
    m = mean_df[i].mean()
    s = mean_df[i].std()
    
    est_list = []

    for j in range(len(nan_zip)):
        
        # Process of estimation
        # Mean + [random(normally distributed) * stddev]
        est_list.append(m + (np.random.normal(0) * s))
    
    # Assigning the list as values to the corresponding key
    estimation[i] = est_list

In [8]:
# Converting to dataframe
est = pd.DataFrame(estimation, index = nan_zip)
est.head()

,Cost_mean,Cost_std,Rate_mean,Rate_std,Review_mean,Review_std,Sentiment_mean
10006,1.809425,0.417065,3.982864,0.485156,127.272038,340.021578,0.235054
10020,1.796132,0.512982,3.753600,0.570361,231.863269,600.941204,0.260160
10034,1.731781,0.540382,4.032106,0.767694,181.627494,68.366410,0.230491
10037,1.617226,0.495061,3.877907,0.802988,-12.628595,115.718393,0.190613
10044,1.722757,0.797353,4.328224,0.341574,102.443671,119.219510,0.222827


In [9]:
# Creating final copy of full yelp metrics for all zipcodes

full_yelp = pd.concat([mean_df, est], axis = 0)
full_yelp.shape

(187, 7)

In [10]:
# Confiming concatenation was done correctly
# Checking an estimated row of concatenated with prior to concatenation
full_yelp.loc[10006, :]

Cost_mean           1.809425
Cost_std            0.417065
Rate_mean           3.982864
Rate_std            0.485156
Review_mean       127.272038
Review_std        340.021578
Sentiment_mean      0.235054
Name: 10006, dtype: float64

## Method 2: Create Yelp values using 4 (numerically) nearest zipcodes

In [46]:
estimation = {}

# defining the upper bound
max_zip = max(mean_df.index)

for i in nan_zip:
    
    ref_list = []
    # Number of neighboring zipcodes to consider
    total = 4

    count_up = 0
    new_zip = i
    
    # Getting 2 from higher zipcodes
    
    while (count_up < (total/2)) and new_zip <= max_zip:

        new_zip += 1
        if new_zip in mean_df.index:
            ref_list.append(new_zip)
            count_up += 1
            new_zip += 1

        else:
            new_zip += 1

    count_down = 0
    new_zip = i
    
    # 2 from lower zipcodes
    while count_down < (total - count_up) and new_zip > 10001:
        new_zip -= 1
        if new_zip in mean_df.index:
            ref_list.append(new_zip)
            count_up += 1
            new_zip -= 1

        else:
            new_zip -= 1
    
    estimation[i] = ref_list

In [58]:
# Create an empty dictionary
vals = {}

for i in estimation:
    
    est_list = []
    
    for j in mean_df.columns:
    
        est_list.append(mean_df[mean_df.index.isin(estimation[i])][j].mean())

    # Assigning the list as values to the corresponding key
    vals[i] = est_list

In [62]:
# Converting to dataframe
neighbour = pd.DataFrame(vals).T
neighbour.columns = mean_df.columns
neighbour.head()

,Cost_mean,Cost_std,Rate_mean,Rate_std,Review_mean,Review_std,Sentiment_mean
10006,1.769508,0.573354,4.091761,0.398775,279.256818,392.982450,0.225003
10020,1.958333,0.550737,4.072917,0.560024,247.775000,281.387446,0.234275
10034,1.762500,0.478383,4.183333,0.463025,105.929167,89.231401,0.239799
10037,1.831250,0.609297,4.084375,0.656403,166.881250,206.088464,0.228812
10044,1.783333,0.496933,4.170833,0.411108,220.466667,197.619452,0.239143


In [63]:
# Join to existing dataframe to complete yelp data for all NYC zipcodes
neighbour_yelp = pd.concat([mean_df, neighbour], axis = 0)
neighbour_yelp.shape

(187, 7)

In [64]:
# Saving the dataframe
with open('/Users/peidacai/Desktop/full_yelp_ref_df.pkl', 'w') as f:
    pickle.dump(full_yelp, f)